In [1]:
import cv2
import numpy as np

In [23]:
# Función para redimensionar una imagen a un tamaño fijo
def resize_image(image, size):
    return cv2.resize(image, size, interpolation=cv2.INTER_AREA)

# Función para leer y convertir las imágenes a escala de grises
def read_images(filenames):
    """Lee las imágenes de los archivos y las convierte a escala de grises."""
    images = [cv2.imread(file) for file in filenames]
    resized_images = [resize_image(img, (512,512)) for img in images]
    images_grey = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in resized_images]

    # DEBUG
    ''' for idx, img in enumerate(images_grey):
        cv2.imshow(f"Image {idx+1}", img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()'''
    return images_grey

# Función para detectar características y descriptores usando SIFT
def detect_features(images_grey):
    """Detecta puntos clave y calcula descriptores para una lista de imágenes en escala de grises."""
    sift = cv2.SIFT_create()
    #keypoints_descriptors = [sift.detectAndCompute(image, None) for image in images_grey]
    keypoints_descriptors = []
    for idx, image in enumerate(images_grey):
        keypoints, descriptor = sift.detectAndCompute(image, None)
        keypoints_descriptors.append((keypoints, descriptor))
        img_with_keypoints = cv2.drawKeypoints(image, keypoints, None)
        cv2.imshow(f"Features Image {idx+1}", img_with_keypoints)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    return keypoints_descriptors

# Función para emparejar descriptores entre dos imágenes
def match_descriptors(descriptor1, descriptor2, ratio_test=0.7):
    # Crear un emparejador de fuerza bruta con la distancia euclidiana
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
    
    # Encuentra los dos mejores matches para cada descriptor
    matches = bf.knnMatch(descriptor1, descriptor2, k=2)

    # Aplicar el test de ratio de Lowe
    good_matches = []
    for m, n in matches:
        if m.distance < ratio_test * n.distance:
            good_matches.append(m)
    return good_matches


def stitch_images(image_left, image_middle, kp_left, kp_middle, matches):
    points_left = np.float32([kp_left[m.queryIdx].pt for m in matches])
    points_middle = np.float32([kp_middle[m.trainIdx].pt for m in matches])

    H, _ = cv2.findHomography(points_left, points_middle, cv2.RANSAC, 5.0)

    # Encontrar las dimensiones de la imagen de la izquierda transformada
    h_left, w_left = image_left.shape
    corners_left = np.float32([[0, 0], [0, h_left], [w_left, h_left], [w_left, 0]]).reshape(-1, 1, 2)
    corners_left_transformed = cv2.perspectiveTransform(corners_left, H)
    [x_min, y_min] = np.int32(corners_left_transformed.min(axis=0).ravel() - 0.5)
    [x_max, y_max] = np.int32(corners_left_transformed.max(axis=0).ravel() + 0.5)
    
    # Ajustar la homografía para traducir la imagen y evitar que se recorte
    translation_dist = [-x_min, -y_min]
    H_translation = np.array([[1, 0, translation_dist[0]], [0, 1, translation_dist[1]], [0, 0, 1]])
    H_modified = H_translation.dot(H)

   # Asegurándonos de que es lo suficientemente grande para contener ambas imágenes
    output_width = max(x_max, image_middle.shape[1] - x_min)
    output_height = max(y_max, image_middle.shape[0] - y_min)
    output_img_size = (output_width, output_height)

    # Aplicamos la homografía con el tamaño de salida calculado
    transformed_left = cv2.warpPerspective(image_left, H_modified, output_img_size)

    # Creamos la imagen combinada con el tamaño de salida
    combined = np.zeros((output_height, output_width), dtype=np.uint8)

    # Aseguramos que el índice no sea negativo
    x_offset = max(translation_dist[0], 0)
    y_offset = max(translation_dist[1], 0)

    # Colocamos la imagen del medio en su nueva posición en la imagen combinada
    combined[y_offset:y_offset+image_middle.shape[0], x_offset:x_offset+image_middle.shape[1]] = image_middle

    # Creamos una máscara para la imagen transformada
    mask = transformed_left > 0

    # Superponemos la imagen transformada sobre la imagen combinada
    combined[mask] = transformed_left[mask]

    return combined

In [25]:
# Nombres de archivo de las imágenes de entrada
H1_filenames = ['input_photos/V1_IZQ.jpeg', 'input_photos/V1_MID.jpeg', 'input_photos/V1_DER.jpeg']
H2_filenames = ['input_photos/V2_IZQ.jpeg', 'input_photos/V2_MID.jpeg', 'input_photos/V2_DER.jpeg']
H3_filenames = ['input_photos/V3_IZQ.jpeg', 'input_photos/V3_MID.jpeg', 'input_photos/V3_DER.jpeg']

V_filenames = ['input_photos/H_IZQ.jpeg', 'input_photos/H_MID.jpeg', 'input_photos/H_DER.jpeg']
all_filenames = [V_filenames, H1_filenames, H2_filenames, H3_filenames]

for filenames in all_filenames:
    # Leemos y convertimos las imágenes a escala de aa
    images_grey = read_images(filenames)

    # Detectamos características en las imágenes en escala de grises
    kps_des = detect_features(images_grey)

    # Emparejamos los descriptores entre la primera y la segunda imagen
    matches12 = match_descriptors(kps_des[0][1], kps_des[1][1])

    # Unimos las primeras dos imágenes
    result_12 = stitch_images(images_grey[0], images_grey[1], kps_des[0][0], kps_des[1][0], matches12)

    # Detectamos características en la imagen resultante de la unión de la primera y segunda imagen
    kp_result_12, desc_result_12 = detect_features([result_12])[0]

    # Detectamos características en la tercera imagen
    kp_image_3, desc_image_3 = detect_features([images_grey[2]])[0]

    # Emparejamos los descriptores entre la imagen compuesta y la tercera imagen
    matches_12_3 = match_descriptors(desc_result_12, desc_image_3)

    # Calculamos la homografía entre la imagen compuesta y la tercera imagen
    points1 = np.zeros((len(matches_12_3), 2), dtype=np.float32)
    points2 = np.zeros_like(points1)

    for i, match in enumerate(matches_12_3):
        points1[i, :] = kp_result_12[match.queryIdx].pt
        points2[i, :] = kp_image_3[match.trainIdx].pt

    H2, _ = cv2.findHomography(points2, points1, cv2.RANSAC, 8.0)

    # Unimos el resultado anterior con la tercera imagen
    height, width = result_12.shape[:2]
    new_width = width + images_grey[2].shape[1]

    result_123 = cv2.warpPerspective(images_grey[2], H2, (new_width, height))
    result_123[0:height, 0:width] = result_12

    # Guardamos y mostramos el resultado
    output_filename = 'results/panoramic_image_resize.png'  # Asegúrate de que esta ruta sea correcta y accesible
    cv2.imwrite(output_filename, result_123)
    resized_img = cv2.resize(result_123, None,  fx=0.5, fy=0.5)
    cv2.imshow("Panoramic Image", resized_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Devolvemos el nombre del archivo de salida

    output_filename